## Alterar se necessário (Definições)

In [51]:
#valor do número de rodadas de cada jogo
valores = [4, 4, 5, 5, 4]

#quantidade total de jogos
qt_jogos = 5

#valor de referência do jogo inicial e final
valor_ini_jogo = 5
valor_fim_jogo = 9

# Importações

In [ ]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Dados_Gerais/venda_aluguel.csv', sep=',')
#df.columns = ['acao', 'pessoa1', 'pessoa2', 'parcela', 'objeto', 'tipo_preco', 'rodada', 'jogo']
#'saldo_CD1', 'saldo_CD2', 'saldo_CD3', 'saldo_AT1', 'saldo_AT2', 'saldo_AT3', 'saldo_EmpAgr', 'saldo_EmpFer', 'saldo_EmpMaq', 'saldo_EmpSem', 'saldo_FiscalAT', 'saldo_FiscalCD', 'saldo_PrefeitoAT', 'saldo_PrefeitoCD', 'poluicao_mundial', 'selo', 'prox_taxa', 'val_taxa', 'tratamento'

In [4]:
#Inclui colunas para normalizar o dataset
serie = pd.Series([])
df['saldo_AgCD1'] = serie
df['saldo_AgCD2'] = serie
df['saldo_AgCD3'] = serie
df['saldo_AgAT1'] = serie
df['saldo_AgAT2'] = serie
df['saldo_AgAT3'] = serie
df['saldo_EmpAgro'] = serie
df['saldo_EmpFert'] = serie
df['saldo_EmpMaq'] = serie
df['saldo_EmpSem'] = serie
df['saldo_FiscalAT'] = serie
df['saldo_FiscalCD'] = serie
df['saldo_PrefeitoAT'] = serie
df['saldo_PrefeitoCD'] = serie
df['poluicao_mundial'] = serie
df['selo'] = serie
df['prox_taxa'] = serie
df['val_taxa'] = serie
df['tratamento'] = serie

<ipython-input-4-201eb5e6b459>:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  serie = pd.Series([])


In [5]:
#Carregamento do restante dos dados
df_acoes = pd.read_csv('Dados_Gerais/acao_ambiental.csv', sep=',')
df_imposto = pd.read_csv('Dados_Gerais/imposto.csv', sep=',')
df_multa = pd.read_csv('Dados_Gerais/multa.csv', sep=',')
df_recebe_selo = pd.read_csv('Dados_Gerais/recebe_selo.csv', sep=',')
df_transferencia = pd.read_csv('Dados_Gerais/transferencia.csv', sep=',')
df_troca_taxa = pd.read_csv('Dados_Gerais/troca_taxa.csv', sep=',')
df_taxa_vigente = pd.read_csv('Dados_Gerais/taxa_vigente.csv', sep=',')

# Tratamentos Iniciais

In [6]:
#Separar venda do aluguel

df['acao'] = df['pessoa1'].apply(lambda x: 'aluguel' if x == 'EmpMaq' else'venda')

In [8]:
df['tipo_preco2'] = df['tipo_preco'].apply(lambda x: '0' if x == 'normal' else ('-5' if x == 'baixo' else ('+5' if x == 'alto' else x)))

In [10]:
df = pd.concat([df, df_acoes, df_imposto, df_multa, df_recebe_selo, df_transferencia, df_troca_taxa])

In [12]:
df.drop(['valor'], inplace=True, axis=1)

# Reorganização da base de dados

In [52]:
for i in range(0, qt_jogos):
    jogos[i] = pd.DataFrame()

In [21]:
#range(jogo inicial, jogo final+1)
for i in range(valor_ini_jogo, valor_fim_jogo+1):
    for y in range(1, (valores[i-qt_jogos]+1)):
        jogos[i-qt_jogos] = pd.concat([jogos[i-qt_jogos],df[(df['jogo'] == i)][df[(df['jogo']== i)]['rodada'] == y]])

In [23]:
#insere os saldos e a poluicao mundial iniciais
for i in range(0, qt_jogos):
    #saldo agricultores
    for k in range (8, 14):
        jogos[i].iat[0, k] = 600
    #saldo empresários
    for k in range (14, 18):
        jogos[i].iat[0, k] = 100
    #saldo fiscais    
    for k in range (18, 20):
        jogos[i].iat[0, k] = 0
    #saldo prefeituras
    for k in range (20, 22):
        jogos[i].iat[0, k] = 1000
    #poluicao mundial
    jogos[i].iat[0, 22] = 20

In [33]:
sem_anterior = ['', 0, 0]
fer_anterior = ''
atlantis = ['AgAT1', 'AgAT2', 'AgAT3', 'EmpMaq', 'EmpSem']
cidadela = ['AgCD1', 'AgCD2', 'AgCD3', 'EmpFert', 'EmpAgro']
colunas = []
for i in jogos[0].columns:
    colunas.append(i)
    
def calcula_acao(acao, linha, index):
    global fer_anterior
    global produtividade_agricultor
    global produtividade
    poluicao = 0
    x = linha
    j = index # indice da lista de jogos
    rodada = jogos[j].iloc[[x]]['rodada'].tolist()[0]
    jogo = jogos[j].iloc[[x]]['jogo'].tolist()[0]
    if x != 0:  # atualiza os saldos e poluição, exceto na primeira acao
        #saldo agricultores
        for k in range (8, 14):
            jogos[j].iat[x, k] = jogos[j].iat[x-1, k]
        #saldo empresários
        for k in range (14, 18):
            jogos[j].iat[x, k] = jogos[j].iat[x-1, k]
        #saldo fiscais    
        for k in range (18, 20):
            jogos[j].iat[x, k] = jogos[j].iat[x-1, k]
        #saldo prefeituras
        for k in range (20, 22):
            jogos[j].iat[x, k] = jogos[j].iat[x-1, k]
        #poluicao mundial
        jogos[j].iat[x, 22] = jogos[j].iat[x-1, 22]
    elif x == 0 and rodada != 0:
        produtividade = [0,0,0,0,0,0,0,0,0,0] # limpa a lista para a proxima rodada
        produtividade_agricultor = 0
    
    if acao == 'venda':
        impostos.clear()
        produto = jogos[j].iloc[[x]]['objeto'].tolist()[0]
        empresario = jogos[j].iloc[[x]]['pessoa1'].tolist()[0]
        agricultor = jogos[j].iloc[[x]]['pessoa2'].tolist()[0]
        tipo_preco = int(jogos[j].iloc[[x]]['tipo_preco2'].tolist()[0])  
        s_pessoa1 = 0
        s_pessoa2 = 0
        
        empresario = empresario.split('|')[0]
        agricultor = agricultor.split('|')[0]
        
        if empresario == 'EmpSem':
            fer_anterior = ''
            sem_anterior[2] = 0
            sem_anterior[1] = 0
            sem_anterior[0] = produto
            if produto == 'Soja':
                s_pessoa1 = 30+tipo_preco
                s_pessoa2 = -(30+tipo_preco)+10
                poluicao = 30+3
                sem_anterior[2] = 30
            elif produto == 'Arroz':
                s_pessoa1 = 20+tipo_preco
                s_pessoa2 = -(20+tipo_preco)+10
                poluicao = 20+2
                sem_anterior[2] = 20
            else:
                #Só pode ser hortaliças!!
                s_pessoa1 = 10+tipo_preco
                s_pessoa2 = -(10+tipo_preco)+10
                poluicao = 10+1
                sem_anterior[2] = 10
            sem_anterior[1] += 10
            produtividade[9] += s_pessoa1 # produtividade do empresario de sementes
            produtividade_agricultor = 10
        elif empresario == 'EmpFer':
            if produto == 'Comum':
                s_pessoa1 = 10+tipo_preco
                s_pessoa2 = -(10+tipo_preco)+20
                sem_anterior[1] += 20
                fer_anterior = 'Comum'
                poluicao = 9
                produtividade_agricultor = 20
            elif produto == 'Premium':
                s_pessoa1 = 20+tipo_preco
                s_pessoa2 = -(20+tipo_preco)+30
                sem_anterior[1] += 30
                fer_anterior = 'Premium'
                poluicao = 6
                produtividade_agricultor = 30
            else:
                #Só pode ser o super premium!!
                s_pessoa1 = 30+tipo_preco
                s_pessoa2 = - (30+tipo_preco)+40
                sem_anterior[1] += 40
                fer_anterior = 'Super premium'
                poluicao = 3
                produtividade_agricultor = 40
            produtividade[7] += s_pessoa1 # produtividade do empresario de fertilizante
        else:
            #Só pode ser o empresario de agrotóxico!!
            if produto == 'Comum':
                s_pessoa1 = 10+tipo_preco
                s_pessoa2 = - (10+tipo_preco)
                if fer_anterior != '':
                    if fer_anterior == 'Comum':
                        s_pessoa2 += 40
                        produtividade_agricultor = 40
                        sem_anterior[1] += 40
                    elif fer_anterior == 'Premium':
                        s_pessoa2 += 60
                        produtividade_agricultor = 60
                        sem_anterior[1] += 60
                    else:
                        s_pessoa2 += 80
                        produtividade_agricultor = 80
                        sem_anterior[1] += 80
                else:
                    s_pessoa2 += 20
                    produtividade_agricultor = 20
                if sem_anterior[0] == 'Hortalica':
                    poluicao = 20
                    sem_anterior[2] += 20
                elif sem_anterior[0] == 'Arroz':
                    poluicao = 40
                    sem_anterior[2] += 40
                else: 
                    poluicao = 60                    
                    sem_anterior[2] += 60
                    sem_anterior[1] = sem_anterior[1]*2
                poluicao += 3
            elif produto == 'Premium':
                s_pessoa1 = 20+tipo_preco
                s_pessoa2 = - (20+tipo_preco)
                if fer_anterior != '':
                    if fer_anterior == 'Comum':
                        s_pessoa2 += 100
                        produtividade_agricultor = 100
                        sem_anterior[1] += 100
                    elif fer_anterior == 'Premium':
                        s_pessoa2 += 150
                        produtividade_agricultor = 150
                        sem_anterior[1] += 150
                    else:
                        s_pessoa2 += 200
                        produtividade_agricultor = 200
                        sem_anterior[1] += 200
                else:
                    s_pessoa2 += 50
                    produtividade_agricultor = 50
                if sem_anterior[0] == 'Hortalica':
                    poluicao = 50
                    sem_anterior[2] += 50
                elif sem_anterior[0] == 'Arroz':
                    poluicao = 100
                    sem_anterior[2] += 100
                else: 
                    poluicao = 150
                    sem_anterior[2] += 150
                    sem_anterior[1] = sem_anterior[1]*2
                poluicao += 2
            else:
                #Só pode ser o super premium!!
                s_pessoa1 = 30+tipo_preco
                s_pessoa2 = - (30+tipo_preco)
                if fer_anterior != '':
                    if fer_anterior == 'Comum':
                        s_pessoa2 += 180
                        produtividade_agricultor = 180
                        sem_anterior[1] += 180
                    elif fer_anterior == 'Premium':
                        s_pessoa2 += 270
                        produtividade_agricultor = 270
                        sem_anterior[1] += 270
                    else:
                        s_pessoa2 += 360
                        produtividade_agricultor = 360
                        sem_anterior[1] += 360
                else:
                    s_pessoa2 += 90
                    produtividade_agricultor = 90
                if sem_anterior[0] == 'Hortalica':
                    poluicao = 90
                    sem_anterior[2] += 90
                elif sem_anterior[0] == 'Arroz':
                    poluicao = 180
                    sem_anterior[2] += 180
                else: 
                    poluicao = 270
                    sem_anterior[2] += 270
                    sem_anterior[1] = sem_anterior[1]*2
                poluicao += 1
            s_pessoa2 += sem_anterior[1]
            produtividade[6] += s_pessoa1 # produtividade do empresario de agrotoxico
        s_pessoa1 = jogos[j].iloc[[x]]['saldo_'+empresario].tolist()[0] + s_pessoa1
        s_pessoa2 = jogos[j].iloc[[x]]['saldo_'+agricultor].tolist()[0] + s_pessoa2
        indice_pessoa1 = colunas.index('saldo_'+empresario)
        indice_pessoa2 = colunas.index('saldo_'+agricultor)
        jogos[j].iat[x, indice_pessoa1] = s_pessoa1
        jogos[j].iat[x, indice_pessoa2] = s_pessoa2
        jogos[j].iat[x, 22] = (jogos[j].iat[x, 22]/100 + poluicao/10000)*100
        

    elif acao == "aluguel":
        s_pessoa1 = 0
        s_pessoa2 = 0
        produto = jogos[j].iloc[[x]]['objeto'].tolist()[0]
        empresario = jogos[j].iloc[[x]]['pessoa1'].tolist()[0]
        agricultor = jogos[j].iloc[[x]]['pessoa2'].tolist()[0]
        tipo_preco = int(jogos[j].iloc[[x]]['tipo_preco2'].tolist()[0])
        
        empresario = empresario.split('|')[0]
        agricultor = agricultor.split('|')[0]
        
        if produto == 'Maquinas 1':
            s_pessoa1 = 30+tipo_preco
            s_pessoa2 = -(30+tipo_preco)
            poluicao = 3
            if fer_anterior != '':
                if fer_anterior == 'Comum':
                    sem_anterior[1] += 40 
                    s_pessoa2 += 40
                    produtividade_agricultor = 40
                elif fer_anterior == 'Premium':
                    sem_anterior[1] += 60
                    s_pessoa2 += 60
                    produtividade_agricultor = 60
                else:
                    sem_anterior[1] += 80
                    s_pessoa2 += 80            
                    produtividade_agricultor = 80
        elif produto == 'Maquinas 2':
            s_pessoa1 = 60+tipo_preco
            s_pessoa2 = -(60+tipo_preco)
            poluicao = 6
            if fer_anterior != '':
                if fer_anterior == 'Comum':
                    sem_anterior[1] += 100 
                    s_pessoa2 += 100
                    produtividade_agricultor = 100
                elif fer_anterior == 'Premium':
                    sem_anterior[1] += 150
                    s_pessoa2 += 150
                    produtividade_agricultor = 150
                else:
                    sem_anterior[1] += 200
                    s_pessoa2 += 200
                    produtividade_agricultor = 200
        elif produto == 'Maquinas 3':
            s_pessoa1 = 90+tipo_preco
            s_pessoa2 = -(90+tipo_preco)
            poluicao = 9
            if fer_anterior != '':
                if fer_anterior == 'Comum':
                    sem_anterior[1] += 180 
                    s_pessoa2 += 180
                    produtividade_agricultor = 180
                elif fer_anterior == 'Premium':
                    sem_anterior[1] += 270
                    s_pessoa2 += 270
                    produtividade_agricultor = 270
                else:
                    sem_anterior[1] += 360
                    s_pessoa2 += 360
                    produtividade_agricultor = 360
        else:
            #É o pulverizador
            s_pessoa1 = 400+tipo_preco
            s_pessoa2 = -(400+tipo_preco)
            poluicao = -sem_anterior[2]/2
            poluicao += 40
        produtividade[8] += s_pessoa1 # produtividade do empresario de maquinas
        s_pessoa1 = jogos[j].iloc[[x]]['saldo_'+empresario].tolist()[0] + s_pessoa1
        s_pessoa2 = jogos[j].iloc[[x]]['saldo_'+agricultor].tolist()[0] + s_pessoa2
        indice_pessoa1 = colunas.index('saldo_'+empresario)
        indice_pessoa2 = colunas.index('saldo_'+agricultor)
        jogos[j].iat[x, indice_pessoa1] = s_pessoa1
        jogos[j].iat[x, indice_pessoa2] = s_pessoa2
        jogos[j].iat[x, 22] = (jogos[j].iat[x, 22]/100 + poluicao/10000)*100

    elif acao == "imposto":
        prefeito = jogos[j].iloc[[x]]['pessoa1'].tolist()[0]
        produtor = jogos[j].iloc[[x]]['pessoa2'].tolist()[0] # empresario ou agricultor
        #prefeito = prefeito.replace("prefeito ", "")
        producao = 0
        
        prefeito = prefeito.split('|')[1]
        produtor = produtor.split('|')[0]
        
        #if prefeito in atlantis:
        #    prefeito = 'PrefeitoAT'
        #else:
        #    prefeito = 'PrefeitoCD'
        if produtor in atlantis:
            cidade = 'Atlantis'
            if produtor == 'AgAT1':
                producao = produtividade[0]
            elif produtor == 'AgAT2':
                producao = produtividade[1]
            elif produtor == 'AgAT3':
                producao = produtividade[2]
            elif produtor == 'EmpMaq':
                producao = produtividade[8]
            else:
                 producao = produtividade[9]                          
        else:
            cidade = 'Cidadela'
            if produtor == 'AgCD1':
                producao = produtividade[3]
            elif produtor == 'AgCD2':
                producao = produtividade[4]
            elif produtor == 'AgCD3':
                producao = produtividade[5]
            elif produtor == 'EmpFert':
                producao = produtividade[7]
            else:
                producao = produtividade[6]        
        if producao == 0:  # baixa produtividada
            tipo_taxa = 1
        elif producao > 0 and producao <= 200: # media produtividada
            tipo_taxa = 2
        else: # alta produtividada
            tipo_taxa = 3
        valor = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['tipo_taxa'] == tipo_taxa)][(df_taxa_vigente['cidade'] == cidade)].iloc[:]['valor'].tolist()[0]
        if '%' in valor:
            valor = valor.replace('%','')
            valor = int(valor)/100
            imposto = valor * producao
        else:
            imposto = int(valor)
        impostos.append(imposto) # para utilizar no desconto de quem conseguiu selo verde
        s_pessoa1 = jogos[j].iloc[[x]]['saldo_'+prefeito].tolist()[0]
        s_pessoa2 = jogos[j].iloc[[x]]['saldo_'+produtor].tolist()[0]
        indice_pessoa1 = colunas.index('saldo_'+prefeito)
        indice_pessoa2 = colunas.index('saldo_'+produtor)
        jogos[j].iat[x, indice_pessoa1] = s_pessoa1 + imposto
        jogos[j].iat[x, indice_pessoa2] = s_pessoa2 - imposto

    elif acao == "multa": # valor da multa vai para a prefeitura
        pessoa1 = jogos[j].iloc[[x]]['pessoa1'].tolist()[0]
        pessoa2 = jogos[j].iloc[[x]]['pessoa2'].tolist()[0]
        
        pessoa1 = pessoa1.split('|')[1]
        pessoa2 = pessoa2.split('|')[0]
        
        s_pessoa2 = jogos[j].iloc[[x]]['saldo_'+pessoa2].tolist()[0]
        valor_multa = jogos[j].iloc[[x]]['valor3'].tolist()[0]       
        
        
        if pessoa1 in atlantis:
            prefeito = 'PrefeitoAT'
        else:
            prefeito = 'PrefeitoCD'
        s_pessoa1 = jogos[j].iloc[[x]]['saldo_'+prefeito].tolist()[0]
        indice_pessoa1 = colunas.index('saldo_'+prefeito)
        indice_pessoa2 = colunas.index('saldo_'+pessoa2)
        jogos[j].iat[x, indice_pessoa1] = float(s_pessoa1) + float(valor_multa)
        jogos[j].iat[x, indice_pessoa2] = float(s_pessoa2) - float(valor_multa)

    elif acao == "selo": # 5% de desconto no imposto por parcela, o dinheiro descontado é o do caixa da prefeitura
        pessoa1 = jogos[j].iloc[[x]]['pessoa1'].tolist()[0] # fiscal
        pessoa2 = jogos[j].iloc[[x]]['pessoa2'].tolist()[0] # agricultor 
        prefeito_atlantis = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['cidade'] == 'Atlantis')].iloc[:]['personagem'].tolist()[0]     
        prefeito_cidadela = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['cidade'] == 'Cidadela')].iloc[:]['personagem'].tolist()[0]
        
        pessoa1 = pessoa1.split('|')[1]
        pessoa2 = pessoa2.split('|')[0]
        
        if pessoa2 in atlantis:
            s_pessoa1 = jogos[j].iloc[[x]]['saldo_PrefeitoAT'].tolist()[0] 
            pessoa1 = 'PrefeitoAT'
        else:
            s_pessoa1 = jogos[j].iloc[[x]]['saldo_PrefeitoCD'].tolist()[0]
            pessoa1 = 'PrefeitoCD'
        s_pessoa2 = jogos[j].iloc[[x]]['saldo_'+pessoa2].tolist()[0]
        if pessoa2 == 'AgAT1':
            desconto = impostos[4]*0.05
        elif pessoa2 == 'AgAT2':
            desconto = impostos[6]*0.05
        elif pessoa2 == 'AgAT3':
            desconto = impostos[8]*0.05
        elif pessoa2 == 'AgCD1':
            desconto = impostos[5]*0.05
        elif pessoa2 == 'AgCD2':
            desconto = impostos[7]*0.05
        elif pessoa2 == 'AgCD3':
            desconto = impostos[9]*0.05    
        s_pessoa1 = s_pessoa1 - desconto # prefeito
        s_pessoa2 = s_pessoa2 + desconto # agricultor
        indice_pessoa1 = colunas.index('saldo_'+pessoa1)
        indice_pessoa2 = colunas.index('saldo_'+pessoa2)
        jogos[j].iat[x, indice_pessoa1] = s_pessoa1
        jogos[j].iat[x, indice_pessoa2] = s_pessoa2

    elif acao == "transferencia":
        pessoa1 = jogos[j].iloc[[x]]['pessoa1'].tolist()[0]
        etapa1 = jogos[j].iloc[[x]]['etapa_1'].tolist()[0]
        pessoa2 = jogos[j].iloc[[x]]['pessoa2'].tolist()[0]
        etapa2 = jogos[j].iloc[[x]]['etapa_2'].tolist()[0]        
        
        pessoa1 = pessoa1.split('|')[int(etapa1)-1]
        pessoa2 = pessoa2.split('|')[int(etapa2)-1]
                 
        s_pessoa1 = jogos[j].iloc[[x]]['saldo_'+pessoa1].tolist()[0]
        s_pessoa2 = jogos[j].iloc[[x]]['saldo_'+pessoa2].tolist()[0]
        valor = jogos[j].iloc[[x]]['valor2'].tolist()[0]
        s_pessoa1 = s_pessoa1 - valor
        s_pessoa2 = s_pessoa2 + valor
        indice_pessoa1 = colunas.index('saldo_'+pessoa1)
        indice_pessoa2 = colunas.index('saldo_'+pessoa2)
        jogos[j].iat[x, indice_pessoa1] = s_pessoa1
        jogos[j].iat[x, indice_pessoa2] = s_pessoa2

    elif acao == "tratamento":
        s_pessoa1 = 0
        tratamento = jogos[j].iloc[[x]]['tratamento'].tolist()[0]
        prefeito = jogos[j].iloc[[x]]['pessoa1'].tolist()[0]      
        poluicao = jogos[j].iloc[[x]]['poluicao_mundial'].tolist()[0]
        p = 0
        reducao = 0
        
        prefeito = prefeito.split('|')[1]

        if poluicao >= 30:            
            p = poluicao - 20
        if tratamento == 'Lixo':
            #reduz 10%
            s_pessoa1 = -1600-p
            reducao = 10
        elif tratamento == 'Agua':
            #reduz 5%
            s_pessoa1 = -800-p
            reducao = 5
        else:
            #tratamento de esgoto 15%
            s_pessoa1 = -2400-p
            reducao = 15

        s_pessoa1 = jogos[j].iloc[[x]]['saldo_'+prefeito].tolist()[0] + s_pessoa1
        indice_pessoa1 = colunas.index('saldo_'+prefeito)
        jogos[j].iat[x, indice_pessoa1] = s_pessoa1
        jogos[j].iat[x, 22] = ((poluicao/100)*(1-(reducao/100)))*100
    
    if acao == 'venda' or acao == 'aluguel': # atualiza a produtividade do agricultor conforme o passar da rodada
        if agricultor == 'AgAT1':
            produtividade[0] += produtividade_agricultor
        elif agricultor == 'AgAT2':
            produtividade[1] += produtividade_agricultor
        elif agricultor == 'AgAT3':
            produtividade[2] += produtividade_agricultor
        elif agricultor == 'AgCD1':
            produtividade[3] += produtividade_agricultor
        elif agricultor == 'AgCD2':
            produtividade[4] += produtividade_agricultor
        else:
            produtividade[5] += produtividade_agricultor

In [44]:
produtividade = [0,0,0,0,0,0,0,0,0,0] # produtividade dos personagens na rodada AT1 AT2 AT3 CD1 CD2 CD3 EmpAgr EmpFer EmpMaq EmpSem
taxa = []
produtividade_agricultor = 0
impostos = []
j = 0
for i in range(0, qt_jogos): # jogos
    linhas = jogos[i].shape[0] # acoes de cada jogo
    for j in range(0, linhas):
        acao = jogos[i].iloc[j, 0]
        calcula_acao(acao, j, i)
    produtividade = [0,0,0,0,0,0,0,0,0,0]
    taxa.clear()
    sem_anterior = ['', 0, 0]
    fer_anterior = ''
    impostos.clear()
    produtividade_agricultor = 0

<ipython-input-33-cf374cbc92d9>:321: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valor = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['tipo_taxa'] == tipo_taxa)][(df_taxa_vigente['cidade'] == cidade)].iloc[:]['valor'].tolist()[0]
<ipython-input-33-cf374cbc92d9>:321: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valor = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['tipo_taxa'] == tipo_taxa)][(df_taxa_vigente['cidade'] == cidade)].iloc[:]['valor'].tolist()[0]
<ipython-input-33-cf374cbc92d9>:321: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valor = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['tipo_taxa'] == tipo_taxa)][(df_taxa_vigente['cidade'] == cidade)].iloc[:]['valor'].tolist()[0]
<ipython-input-33-cf37

<ipython-input-33-cf374cbc92d9>:321: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valor = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['tipo_taxa'] == tipo_taxa)][(df_taxa_vigente['cidade'] == cidade)].iloc[:]['valor'].tolist()[0]
<ipython-input-33-cf374cbc92d9>:321: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valor = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['tipo_taxa'] == tipo_taxa)][(df_taxa_vigente['cidade'] == cidade)].iloc[:]['valor'].tolist()[0]
<ipython-input-33-cf374cbc92d9>:321: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valor = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['tipo_taxa'] == tipo_taxa)][(df_taxa_vigente['cidade'] == cidade)].iloc[:]['valor'].tolist()[0]
<ipython-input-33-cf37

<ipython-input-33-cf374cbc92d9>:321: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valor = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['tipo_taxa'] == tipo_taxa)][(df_taxa_vigente['cidade'] == cidade)].iloc[:]['valor'].tolist()[0]
<ipython-input-33-cf374cbc92d9>:321: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valor = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['tipo_taxa'] == tipo_taxa)][(df_taxa_vigente['cidade'] == cidade)].iloc[:]['valor'].tolist()[0]
<ipython-input-33-cf374cbc92d9>:321: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valor = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['tipo_taxa'] == tipo_taxa)][(df_taxa_vigente['cidade'] == cidade)].iloc[:]['valor'].tolist()[0]
<ipython-input-33-cf37

<ipython-input-33-cf374cbc92d9>:360: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prefeito_atlantis = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['cidade'] == 'Atlantis')].iloc[:]['personagem'].tolist()[0]
<ipython-input-33-cf374cbc92d9>:361: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prefeito_cidadela = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['cidade'] == 'Cidadela')].iloc[:]['personagem'].tolist()[0]
<ipython-input-33-cf374cbc92d9>:360: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prefeito_atlantis = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['cidade'] == 'Atlantis')].iloc[:]['personagem'].tolist()[0]
<ipython-input-33-cf374cbc92d9>:361: UserWarning: Boolean Series key will be reindexed to matc

<ipython-input-33-cf374cbc92d9>:321: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valor = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['tipo_taxa'] == tipo_taxa)][(df_taxa_vigente['cidade'] == cidade)].iloc[:]['valor'].tolist()[0]
<ipython-input-33-cf374cbc92d9>:321: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valor = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['tipo_taxa'] == tipo_taxa)][(df_taxa_vigente['cidade'] == cidade)].iloc[:]['valor'].tolist()[0]
<ipython-input-33-cf374cbc92d9>:321: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valor = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['tipo_taxa'] == tipo_taxa)][(df_taxa_vigente['cidade'] == cidade)].iloc[:]['valor'].tolist()[0]
<ipython-input-33-cf37

<ipython-input-33-cf374cbc92d9>:361: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prefeito_cidadela = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['cidade'] == 'Cidadela')].iloc[:]['personagem'].tolist()[0]
<ipython-input-33-cf374cbc92d9>:360: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prefeito_atlantis = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['cidade'] == 'Atlantis')].iloc[:]['personagem'].tolist()[0]
<ipython-input-33-cf374cbc92d9>:361: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prefeito_cidadela = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['cidade'] == 'Cidadela')].iloc[:]['personagem'].tolist()[0]
<ipython-input-33-cf374cbc92d9>:360: UserWarning: Boolean Series key will be reindexed to matc

<ipython-input-33-cf374cbc92d9>:321: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valor = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['tipo_taxa'] == tipo_taxa)][(df_taxa_vigente['cidade'] == cidade)].iloc[:]['valor'].tolist()[0]
<ipython-input-33-cf374cbc92d9>:321: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valor = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['tipo_taxa'] == tipo_taxa)][(df_taxa_vigente['cidade'] == cidade)].iloc[:]['valor'].tolist()[0]
<ipython-input-33-cf374cbc92d9>:321: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valor = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['tipo_taxa'] == tipo_taxa)][(df_taxa_vigente['cidade'] == cidade)].iloc[:]['valor'].tolist()[0]
<ipython-input-33-cf37

<ipython-input-33-cf374cbc92d9>:321: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valor = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['tipo_taxa'] == tipo_taxa)][(df_taxa_vigente['cidade'] == cidade)].iloc[:]['valor'].tolist()[0]
<ipython-input-33-cf374cbc92d9>:321: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valor = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['tipo_taxa'] == tipo_taxa)][(df_taxa_vigente['cidade'] == cidade)].iloc[:]['valor'].tolist()[0]
<ipython-input-33-cf374cbc92d9>:321: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valor = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['tipo_taxa'] == tipo_taxa)][(df_taxa_vigente['cidade'] == cidade)].iloc[:]['valor'].tolist()[0]
<ipython-input-33-cf37

<ipython-input-33-cf374cbc92d9>:360: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prefeito_atlantis = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['cidade'] == 'Atlantis')].iloc[:]['personagem'].tolist()[0]
<ipython-input-33-cf374cbc92d9>:361: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prefeito_cidadela = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['cidade'] == 'Cidadela')].iloc[:]['personagem'].tolist()[0]
<ipython-input-33-cf374cbc92d9>:360: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prefeito_atlantis = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['cidade'] == 'Atlantis')].iloc[:]['personagem'].tolist()[0]
<ipython-input-33-cf374cbc92d9>:361: UserWarning: Boolean Series key will be reindexed to matc

<ipython-input-33-cf374cbc92d9>:360: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prefeito_atlantis = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['cidade'] == 'Atlantis')].iloc[:]['personagem'].tolist()[0]
<ipython-input-33-cf374cbc92d9>:361: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prefeito_cidadela = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['cidade'] == 'Cidadela')].iloc[:]['personagem'].tolist()[0]
<ipython-input-33-cf374cbc92d9>:360: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prefeito_atlantis = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['cidade'] == 'Atlantis')].iloc[:]['personagem'].tolist()[0]
<ipython-input-33-cf374cbc92d9>:361: UserWarning: Boolean Series key will be reindexed to matc

In [45]:
df_final = pd.DataFrame([])
for i in range(0, qt_jogos):
    df_final = pd.concat([df_final, jogos[i]])
df_final.drop(['valor2'], inplace=True, axis=1)
df_final.drop(['valor3'], inplace=True, axis=1)
df_final.drop(['tipo_preco2'], inplace=True, axis=1)

2473
2473


# Dataset organizado + sentimentos classificados

In [ ]:
def busca_transacao(acao, emp, agro, rodada, jogo):
    if acao == 'venda':
        df_sent_classificado = open('../ClassificadordeSentimentos/sentimentos_classificados.txt', 'r', encoding='utf8')
        for item in df_sent_classificado:
            aux = item.split(',')
            #print('df base ---' + 'Emp ' + emp + '; Agro ' + agro + '; Rodada ' + str(rodada) + '; Jogo ' + str(jogo))
            #print('df sent ---' + 'Emp ' + aux[4] + '; Agro ' + aux[5] + '; Rodada ' + str(aux[2]) + '; Jogo ' + str(aux[1]) + '\n')
            if (int(aux[1]) == int(jogo) and int(aux[2]) == int(rodada) and aux[4] == emp and aux[5] == agro):
                #print("MATCH")
                return aux[6]
    return np.nan

In [ ]:
df_final['sentimento'] = df_final.apply(lambda x: busca_transacao(x['acao'], x['pessoa1'], x['pessoa2'], x['rodada'], x['jogo']),axis=1)

In [ ]:
df_final.to_csv('df-final.csv', sep='\t' )